# GRMP Attack Experiment - Google Colab

This notebook runs the Graph Representation-based Model Poisoning (GRMP) attack experiment on AG News dataset.

**Paper**: Graph Representation-based Model Poisoning on the Heterogeneous Internet of Agents

## Setup Instructions

1. **Enable GPU**: Runtime → Change runtime type → GPU
2. **Fetch Code**: Run **Step 0** to clone/download the repo if only this notebook was uploaded.
3. **Run all cells**: Runtime → Run all
4. **View results**: Check the `results/` folder for outputs and visualizations



## Step 0: Fetch Code
If you only uploaded this notebook, run this to clone the repository and set the working directory.
If you've already uploaded the Python files, it will reuse them without cloning.


In [ ]:
# Fetch repository and set working directory
import os, sys, subprocess
from pathlib import Path

REPO_URL = 'https://github.com/GuangLun2000/IoA-Attack-GRMP.git'
REPO_DIR = Path('IoA-Attack-GRMP')

def code_files_present():
    return Path('main.py').exists() and Path('client.py').exists()

if code_files_present():
    print('✅ Code files found in current directory.')
else:
    if REPO_DIR.exists():
        print(f'🔁 Using existing folder: {REPO_DIR}')
    else:
        print(f'📥 Cloning {REPO_URL} ...')
        subprocess.run(['git', 'clone', '--depth', '1', REPO_URL], check=True)
    os.chdir(REPO_DIR)
    print(f"✅ Switched to {Path('.').resolve()}")

# Ensure current path is importable for subsequent cells
sys.path.append(str(Path('.').resolve()))
print(f"📂 Working directory: {Path('.').resolve()}")



## Step 1: Install Dependencies


In [ ]:
# Install required packages
from pathlib import Path
req = Path('requirements.txt')
if req.exists():
    print('Installing from requirements.txt ...')
    %pip install -q -r requirements.txt
else:
    print('requirements.txt not found; installing explicit package list...')
    %pip install -q torch>=2.0.0 transformers>=4.35.0 datasets>=2.0.0 numpy>=1.21.0 scikit-learn>=1.0.0 pandas>=1.3.0 tqdm>=4.62.0 matplotlib>=3.4.0 seaborn>=0.11.0

print('✅ Dependencies installed successfully!')


## Step 2: Verify Files and GPU


In [ ]:
# Check if files exist
import os
from pathlib import Path

required_files = ['main.py', 'client.py', 'server.py', 'data_loader.py', 'models.py', 'visualization.py']
missing_files = [f for f in required_files if not os.path.exists(f)]

if missing_files:
    print(f"⚠️  Missing files: {missing_files}")
    print("Please upload these files to Colab using the file uploader.")
else:
    print("✅ All required files found!")
    for f in required_files:
        print(f"  - {f}")

# Check GPU
import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️  No GPU detected. Training will be slower.")
    print("   Go to Runtime → Change runtime type → GPU")


## Step 3: Run Experiment

**Note**: All experiment parameters are configured in `main.py`. 
To modify parameters, edit the `config` dictionary in `main.py` before running this cell.


In [ ]:
# All experiment parameters are configured in main.py
# The experiment will use the config dictionary defined in main.py's main() function
print("✅ Using configuration from main.py")
print("   To modify parameters, edit the 'config' dictionary in main.py")

In [ ]:
# Run the experiment using configuration from main.py
import sys
import warnings
warnings.filterwarnings('ignore')

# Import and run main() function which contains all configuration
from main import main

print("🚀 Starting GRMP Attack Experiment...")
print("=" * 60)
print("Using configuration from main.py")
print("=" * 60)

try:
    # main() function will handle everything: setup, run, and visualization
    main()
    
    print("\n✅ Experiment completed successfully!")
except Exception as e:
    print(f"\n❌ Experiment failed: {e}")
    import traceback
    traceback.print_exc()


## Step 4: View Results and Visualizations

In [ ]:
# Display visualization plots
from IPython.display import Image, display
from pathlib import Path

results_dir = Path("results")
# Use default experiment name from main.py (can be changed in main.py config)
experiment_name = 'vgae_grmp_attack'

# List of all figures to display (4 figures total)
figures = [
    ("Figure 1: Global Accuracy and Stability", f"{experiment_name}_figure1.png"),
    ("Figure 2: Cosine Similarity", f"{experiment_name}_figure2.png"),
    ("Figure 3: Local Accuracy (With Attack)", f"{experiment_name}_figure3.png"),
    ("Figure 4: Euclidean Distance", f"{experiment_name}_figure4.png"),
]

print("📊 Displaying Visualization Figures:")
print("=" * 60)

for fig_title, fig_name in figures:
    fig_path = results_dir / fig_name
    if fig_path.exists():
        print(f"\n✅ {fig_title}")
        display(Image(str(fig_path)))
    else:
        print(f"\n⚠️  {fig_title} not found at {fig_path}")


In [ ]:
# Load and display experiment results summary with enhanced insights
import json
from pathlib import Path
import numpy as np

results_dir = Path("results")
# Use default experiment name from main.py (can be changed in main.py config)
experiment_name = 'vgae_grmp_attack'
results_path = results_dir / f"{experiment_name}_results.json"

if results_path.exists():
    with open(results_path, 'r') as f:
        results_data = json.load(f)
    
    print("📊 Experiment Results Summary:")
    print("=" * 80)
    
    # Display key metrics (using actual metrics from progressive_metrics)
    progressive = results_data.get('progressive_metrics', {})
    rounds = progressive.get('rounds', [])
    clean_acc = progressive.get('clean_acc', [])
    acc_diff = progressive.get('acc_diff', [])
    rejection_rate = progressive.get('rejection_rate', [])
    agg_update_norm = progressive.get('agg_update_norm', [])
    log_data = results_data.get('results', [])
    
    if rounds and clean_acc:
        print(f"\n🎯 Overall Performance Metrics:")
        print("-" * 80)
        print(f"  Total Rounds: {len(rounds)}")
        print(f"  Initial Clean Accuracy: {clean_acc[0]:.4f} ({clean_acc[0]*100:.2f}%)")
        print(f"  Final Clean Accuracy: {clean_acc[-1]:.4f} ({clean_acc[-1]*100:.2f}%)")
        print(f"  Accuracy Change: {clean_acc[-1] - clean_acc[0]:+.4f} ({(clean_acc[-1] - clean_acc[0])*100:+.2f}%)")
        print(f"  Best Clean Accuracy: {max(clean_acc):.4f} ({max(clean_acc)*100:.2f}%) [Round {rounds[clean_acc.index(max(clean_acc))]}]")
        print(f"  Worst Clean Accuracy: {min(clean_acc):.4f} ({min(clean_acc)*100:.2f}%) [Round {rounds[clean_acc.index(min(clean_acc))]}]")
        print(f"  Average Clean Accuracy: {np.mean(clean_acc):.4f} ({np.mean(clean_acc)*100:.2f}%)")
        print(f"  Std Dev of Clean Accuracy: {np.std(clean_acc):.4f} ({np.std(clean_acc)*100:.2f}%)")
        
        if acc_diff:
            print(f"\n  📈 Stability Metrics:")
            print(f"    Final |Δacc| (Stability): {acc_diff[-1]:.4f}")
            print(f"    Peak |Δacc|: {max(acc_diff):.4f} [Round {rounds[acc_diff.index(max(acc_diff))]}]")
            print(f"    Average |Δacc|: {np.mean(acc_diff):.4f}")
            print(f"    Std Dev of |Δacc|: {np.std(acc_diff):.4f}")
        
        if rejection_rate:
            total_rejections = sum(rejection_rate)
            max_rejection_round = rounds[rejection_rate.index(max(rejection_rate))] if rejection_rate else None
            print(f"\n  🛡️  Defense Mechanism Statistics:")
            print(f"    Final Rejection Rate: {rejection_rate[-1]:.4f} ({rejection_rate[-1]*100:.2f}%)")
            print(f"    Average Rejection Rate: {np.mean(rejection_rate):.4f} ({np.mean(rejection_rate)*100:.2f}%)")
            print(f"    Max Rejection Rate: {max(rejection_rate):.4f} ({max(rejection_rate)*100:.2f}%) [Round {max_rejection_round}]")
            print(f"    Total Rounds with Rejections: {sum(1 for r in rejection_rate if r > 0)}/{len(rejection_rate)}")
        
        if agg_update_norm:
            print(f"\n  📐 Update Statistics:")
            print(f"    Final Aggregated Update Norm: {agg_update_norm[-1]:.6f}")
            print(f"    Average Update Norm: {np.mean(agg_update_norm):.6f}")
            print(f"    Max Update Norm: {max(agg_update_norm):.6f} [Round {rounds[agg_update_norm.index(max(agg_update_norm))]}]")
            print(f"    Min Update Norm: {min(agg_update_norm):.6f} [Round {rounds[agg_update_norm.index(min(agg_update_norm))]}]")
        
        # Enhanced defense statistics from log_data
        if log_data:
            print(f"\n  🔍 Defense Mechanism Detailed Analysis:")
            all_similarities = []
            all_thresholds = []
            all_mean_sims = []
            all_std_sims = []
            total_accepted = 0
            total_rejected = 0
            
            for log in log_data:
                defense = log.get('defense', {})
                similarities = defense.get('similarities', [])
                threshold = defense.get('threshold', 0.0)
                mean_sim = defense.get('mean_similarity', 0.0)
                std_sim = defense.get('std_similarity', 0.0)
                accepted = len(defense.get('accepted_clients', []))
                rejected = len(defense.get('rejected_clients', []))
                
                if similarities:
                    all_similarities.extend(similarities)
                if threshold is not None:
                    all_thresholds.append(threshold)
                if mean_sim is not None:
                    all_mean_sims.append(mean_sim)
                if std_sim is not None:
                    all_std_sims.append(std_sim)
                total_accepted += accepted
                total_rejected += rejected
            
            if all_similarities:
                print(f"    Similarity Statistics (all clients, all rounds):")
                print(f"      Mean Similarity: {np.mean(all_similarities):.4f}")
                print(f"      Std Dev: {np.std(all_similarities):.4f}")
                print(f"      Min: {np.min(all_similarities):.4f}")
                print(f"      Max: {np.max(all_similarities):.4f}")
            
            if all_thresholds:
                print(f"    Threshold Statistics:")
                print(f"      Mean Threshold: {np.mean(all_thresholds):.4f}")
                print(f"      Min Threshold: {np.min(all_thresholds):.4f}")
                print(f"      Max Threshold: {np.max(all_thresholds):.4f}")
            
            if all_mean_sims:
                print(f"    Mean Similarity per Round:")
                print(f"      Average: {np.mean(all_mean_sims):.4f}")
                print(f"      Std Dev: {np.std(all_mean_sims):.4f}")
            
            print(f"    Client Acceptance Statistics:")
            print(f"      Total Accepted: {total_accepted}")
            print(f"      Total Rejected: {total_rejected}")
            if total_accepted + total_rejected > 0:
                acceptance_rate = total_accepted / (total_accepted + total_rejected)
                print(f"      Overall Acceptance Rate: {acceptance_rate:.4f} ({acceptance_rate*100:.2f}%)")
        
        # Display ALL per-round summary (no skipping)
        print("\n📈 Per-Round Summary (All Rounds):")
        print("-" * 80)
        print(f"{'Round':<7} | {'Clean Acc':<11} | {'|Δacc|':<9} | {'Rej Rate':<11} | {'Update Norm':<13} | {'Threshold':<11}")
        print("-" * 80)
        for i, r in enumerate(rounds):
            acc = clean_acc[i] if i < len(clean_acc) else 0.0
            diff = acc_diff[i] if i < len(acc_diff) else 0.0
            rej = rejection_rate[i] if i < len(rejection_rate) else 0.0
            norm = agg_update_norm[i] if i < len(agg_update_norm) else 0.0
            threshold = log_data[i].get('defense', {}).get('threshold', 0.0) if i < len(log_data) else 0.0
            print(f"{r:<7} | {acc:<11.4f} | {diff:<9.4f} | {rej:<11.4f} | {norm:<13.6f} | {threshold:<11.4f}")
    else:
        print("⚠️  No progressive metrics found in results")
    
    # Enhanced local accuracies display
    if 'local_accuracies' in results_data and results_data['local_accuracies']:
        local_accs = results_data['local_accuracies']
        print("\n📊 Local Accuracies Analysis:")
        print("-" * 80)
        print(f"{'Client ID':<12} | {'Initial':<11} | {'Final':<11} | {'Change':<11} | {'Average':<11} | {'Std Dev':<11} | {'Best':<11} | {'Worst':<11}")
        print("-" * 80)
        
        for client_id in sorted(local_accs.keys()):
            accs = local_accs[client_id]
            if accs:
                initial = accs[0]
                final = accs[-1]
                change = final - initial
                avg = np.mean(accs)
                std = np.std(accs)
                best = max(accs)
                worst = min(accs)
                print(f"Client {client_id:<6} | {initial:<11.4f} | {final:<11.4f} | {change:>+11.4f} | {avg:<11.4f} | {std:<11.4f} | {best:<11.4f} | {worst:<11.4f}")
        
        # Additional insights
        print(f"\n  📈 Cross-Client Statistics:")
        all_final_accs = [accs[-1] for accs in local_accs.values() if accs]
        all_initial_accs = [accs[0] for accs in local_accs.values() if accs]
        all_avg_accs = [np.mean(accs) for accs in local_accs.values() if accs]
        
        if all_final_accs:
            print(f"    Final Accuracy - Mean: {np.mean(all_final_accs):.4f}, Std: {np.std(all_final_accs):.4f}")
            print(f"    Final Accuracy - Best Client: {max(all_final_accs):.4f}, Worst: {min(all_final_accs):.4f}")
            print(f"    Final Accuracy - Range: {max(all_final_accs) - min(all_final_accs):.4f}")
        
        if all_initial_accs:
            print(f"    Initial Accuracy - Mean: {np.mean(all_initial_accs):.4f}, Std: {np.std(all_initial_accs):.4f}")
        
        if all_avg_accs:
            print(f"    Average Accuracy (all rounds) - Mean: {np.mean(all_avg_accs):.4f}, Std: {np.std(all_avg_accs):.4f}")
    

# Display full configuration summary (all parameters from main.py config)
    if 'config' in results_data:
        config = results_data['config']
        print("\n⚙️  Complete Experiment Configuration:")
        print("=" * 80)
        
        # Helper function to format config values
        def format_value(v):
            if v is None:
                return 'None'
            elif isinstance(v, bool):
                return str(v)
            elif isinstance(v, float):
                return f"{v:.6f}".rstrip('0').rstrip('.') if v != int(v) else f"{int(v)}.0"
            elif isinstance(v, int):
                return str(v)
            else:
                return str(v)
        
        # 1. Experiment Configuration
        print("\n📋 Experiment Configuration:")
        print(f"  experiment_name: {format_value(config.get('experiment_name', 'N/A'))}")
        print(f"  seed: {format_value(config.get('seed', 'N/A'))}")
        
        # 2. Federated Learning Setup
        print("\n👥 Federated Learning Setup:")
        print(f"  num_clients: {format_value(config.get('num_clients', 'N/A'))}")
        print(f"  num_attackers: {format_value(config.get('num_attackers', 'N/A'))}")
        print(f"  num_benign_clients: {format_value(config.get('num_benign_clients', 'N/A'))}")
        print(f"  num_rounds: {format_value(config.get('num_rounds', 'N/A'))}")
        
        # 3. Training Mode Configuration
        print("\n🔧 Training Mode Configuration:")
        print(f"  use_lora: {format_value(config.get('use_lora', 'N/A'))}")
        if config.get('use_lora', False):
            print(f"  lora_r: {format_value(config.get('lora_r', 'N/A'))}")
            print(f"  lora_alpha: {format_value(config.get('lora_alpha', 'N/A'))}")
            print(f"  lora_dropout: {format_value(config.get('lora_dropout', 'N/A'))}")
            print(f"  lora_target_modules: {format_value(config.get('lora_target_modules', 'N/A'))}")
        print(f"  model_name: {format_value(config.get('model_name', 'N/A'))}")
        print(f"  num_labels: {format_value(config.get('num_labels', 'N/A'))}")
        
        # 4. Training Hyperparameters
        print("\n📊 Training Hyperparameters:")
        print(f"  client_lr: {format_value(config.get('client_lr', 'N/A'))}")
        print(f"  server_lr: {format_value(config.get('server_lr', 'N/A'))}")
        print(f"  batch_size: {format_value(config.get('batch_size', 'N/A'))}")
        print(f"  test_batch_size: {format_value(config.get('test_batch_size', 'N/A'))}")
        print(f"  local_epochs: {format_value(config.get('local_epochs', 'N/A'))}")
        print(f"  alpha (FedProx): {format_value(config.get('alpha', 'N/A'))}")
        
        # 5. Data Distribution
        print("\n📦 Data Distribution:")
        print(f"  dirichlet_alpha: {format_value(config.get('dirichlet_alpha', 'N/A'))}")
        print(f"  dataset_size_limit: {format_value(config.get('dataset_size_limit', 'N/A'))}")
        
        # 6. Attack Configuration
        print("\n🎭 Attack Configuration:")
        print(f"  attack_start_round: {format_value(config.get('attack_start_round', 'N/A'))}")
        
        # 7. Formula 4 Constraint Parameters
        print("\n📐 Formula 4 Constraint Parameters:")
        print(f"  d_T: {format_value(config.get('d_T', 'N/A'))}  # Distance threshold for constraint (4b)")
        print(f"  gamma: {format_value(config.get('gamma', 'N/A'))}  # Upper bound for constraint (4c)")
        
        # 8. Lagrangian Dual Parameters
        print("\n🔗 Lagrangian Dual Parameters:")
        print(f"  use_lagrangian_dual: {format_value(config.get('use_lagrangian_dual', 'N/A'))}")
        print(f"  lambda_init: {format_value(config.get('lambda_init', 'N/A'))}")
        print(f"  rho_init: {format_value(config.get('rho_init', 'N/A'))}")
        print(f"  lambda_lr: {format_value(config.get('lambda_lr', 'N/A'))}")
        print(f"  rho_lr: {format_value(config.get('rho_lr', 'N/A'))}")
        
        # 9. VGAE Training Parameters
        print("\n🧠 VGAE Training Parameters:")
        print(f"  dim_reduction_size: {format_value(config.get('dim_reduction_size', 'N/A'))}")
        print(f"  vgae_epochs: {format_value(config.get('vgae_epochs', 'N/A'))}")
        print(f"  vgae_lr: {format_value(config.get('vgae_lr', 'N/A'))}")
        print(f"  vgae_hidden_dim: {format_value(config.get('vgae_hidden_dim', 'N/A'))}")
        print(f"  vgae_latent_dim: {format_value(config.get('vgae_latent_dim', 'N/A'))}")
        print(f"  vgae_dropout: {format_value(config.get('vgae_dropout', 'N/A'))}")
        
        # 10. Attack Optimization Parameters
        print("\n⚡ Attack Optimization Parameters:")
        print(f"  proxy_step: {format_value(config.get('proxy_step', 'N/A'))}")
        print(f"  proxy_steps: {format_value(config.get('proxy_steps', 'N/A'))}")
        print(f"  gsp_perturbation_scale: {format_value(config.get('gsp_perturbation_scale', 'N/A'))}")
        print(f"  opt_init_perturbation_scale: {format_value(config.get('opt_init_perturbation_scale', 'N/A'))}")
        print(f"  grad_clip_norm: {format_value(config.get('grad_clip_norm', 'N/A'))}")
        print(f"  attacker_claimed_data_size: {format_value(config.get('attacker_claimed_data_size', 'N/A'))}")
        
        # 11. Proxy Loss Estimation Parameters
        print("\n📉 Proxy Loss Estimation Parameters:")
        print(f"  proxy_sample_size: {format_value(config.get('proxy_sample_size', 'N/A'))}")
        print(f"  proxy_max_batches_opt: {format_value(config.get('proxy_max_batches_opt', 'N/A'))}")
        print(f"  proxy_max_batches_eval: {format_value(config.get('proxy_max_batches_eval', 'N/A'))}")
        
        # 12. Graph Construction Parameters
        print("\n🕸️  Graph Construction Parameters:")
        print(f"  graph_threshold: {format_value(config.get('graph_threshold', 'N/A'))}")
        
        # 13. Defense Mechanism Parameters
        print("\n🛡️  Defense Mechanism Parameters:")
        print(f"  enable_defense: {format_value(config.get('enable_defense', 'N/A'))}")
        print(f"  defense_threshold: {format_value(config.get('defense_threshold', 'N/A'))}")
        print(f"  tolerance_factor: {format_value(config.get('tolerance_factor', 'N/A'))}")
        print(f"  similarity_alpha: {format_value(config.get('similarity_alpha', 'N/A'))}")
        print(f"  defense_high_rejection_threshold: {format_value(config.get('defense_high_rejection_threshold', 'N/A'))}")
        print(f"  defense_threshold_decay: {format_value(config.get('defense_threshold_decay', 'N/A'))}")
        
        # 14. Visualization
        print("\n📈 Visualization:")
        print(f"  generate_plots: {format_value(config.get('generate_plots', 'N/A'))}")
        print(f"  run_both_experiments: {format_value(config.get('run_both_experiments', 'N/A'))}")
        
        print("\n" + "=" * 80)
else:
    print(f"⚠️  Results file not found: {results_path}")
    print("   Make sure Step 4 has completed successfully.")

## Step 5: Download Results


In [ ]:
# Create a zip file with all results
import zipfile
from pathlib import Path

results_dir = Path("results")
zip_path = "grmp_experiment_results.zip"

if results_dir.exists():
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file_path in results_dir.rglob('*'):
            if file_path.is_file():
                zipf.write(file_path, file_path.relative_to(results_dir.parent))
    
    print(f"✅ Created zip file: {zip_path}")
    print(f"\n📥 Download the file using the cell below")
else:
    print("⚠️  Results directory not found.")


In [ ]:
# Download results zip file
from google.colab import files

if Path("grmp_experiment_results.zip").exists():
    files.download('grmp_experiment_results.zip')
    print("✅ Download started!")
else:
    print("⚠️  Zip file not found. Run the previous cell first.")

## Step 6: Exit Running Time


In [ ]:
from google.colab import runtime
import time

print("✅ All done. All tasks completed.")

runtime.unassign()